In [1]:
import numpy as np
from scipy import sparse
import pandas as pd
import itertools
import hetio.readwrite
from hetio.matrix import metaedge_to_adjacency_matrix

In [2]:
import sys
sys.path.insert(0, '../')
from hetmech.degree_weight import dwpc

In [3]:
repo_url = 'https://github.com/dhimmel/hetionet'
commit = '6d26d15e9055b33b4fd97a180fa288e4f2060b96'
names = ['hetionet-v1.0'] + [f'hetionet-v1.0-perm-{i + 1}' for i in range(5)]    
paths = ['hetnet/json/hetionet-v1.0.json.bz2'] + [
    f'hetnet/permuted/json/{name}.json.bz2' for name in names[1:]
]

In [4]:
metapaths = None

In [5]:
permuted = pd.DataFrame()
for name, path in zip(names, paths):
    url = f'{repo_url}/raw/{commit}/{path}'
    graph = hetio.readwrite.read_graph(url)
    print(f'Loaded {name}')
    if not metapaths:
        metaedge = graph.metagraph.metapath_from_abbrev('CtD')
        metapaths = graph.metagraph.extract_metapaths(source=metaedge.source(), target=metaedge.target(), 
                                                      max_length=1)
    for metapath in metapaths:
        c, d, dwpc_matrix, t = dwpc(graph, metapath)
        print(f'metapath: {metapath}, time: {t}')
        
        out_degrees = np.sum(metaedge_to_adjacency_matrix(graph, metapath[0])[2], axis=1)
        in_degrees = np.sum(metaedge_to_adjacency_matrix(graph, metapath[-1])[2], axis=0)
        degree_combos = list(itertools.product(out_degrees, in_degrees))
        indices = list(itertools.product(range(len(out_degrees)), range(len(in_degrees))))
        index_df = pd.DataFrame(indices, columns=['row', 'col'])
        degree_combo_df = pd.DataFrame(degree_combos, columns=['source_degree', 'target_degree'])
        degree_index_df = index_df.join(degree_combo_df)
                
        # Add DWPC values
        degree_index_df['dwpc'] = degree_index_df.apply(lambda row: dwpc_matrix[row[0], row[1]], axis=1)
        degree_index_df['metapath'] = pd.Series([metapath] * len(degree_index_df))
        degree_index_df.drop(columns=['row', 'col'])
        permuted = permuted.append(degree_index_df)

Loaded hetionet-v1.0
metapath: CtD, time: 0.09397311300199362
metapath: CpD, time: 0.0932347199996002
Loaded hetionet-v1.0-perm-1
metapath: CtD, time: 0.09740564199819346
metapath: CpD, time: 0.09455362300286652
Loaded hetionet-v1.0-perm-2
metapath: CtD, time: 0.1075644660013495
metapath: CpD, time: 0.10089776799941319
Loaded hetionet-v1.0-perm-3
metapath: CtD, time: 0.10227613900133292
metapath: CpD, time: 0.09998473100131378
Loaded hetionet-v1.0-perm-4
metapath: CtD, time: 0.10445344100298826
metapath: CpD, time: 0.10220430300250882
Loaded hetionet-v1.0-perm-5
metapath: CtD, time: 0.10896657899866113
metapath: CpD, time: 0.1023447000006854


## A single metapath:

In [6]:
set(degree_index_df['metapath'])

{CpD}

In [7]:
degree_grouped_permutations = pd.DataFrame(degree_index_df
                                           .groupby(['source_degree', 'target_degree'])['dwpc']
                                           .apply(list)).unstack()

In [8]:
degree_grouped_permutations.head(2)

dwpc  \
target_degree                                                 0    
source_degree                                                      
0              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                                                  \
target_degree                                                 1    
source_degree                                                      
0              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                                                  \
target_degree                                                 2    
source_degree                                                      
0              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                                                  \
target_degree                                                 3    
source_degree                                                      
0              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                                                  \
target_degree                                                 4    
source_degree                                                      
0              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                                                  \
target_degree                                                 5    
source_degree                                                      
0              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                                                  \
target_degree                                                 6    
source_degree                                                      
0              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                                                  \
target_degree                                                 7    
source_degree                                                      
0              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                                                  \
target_degree                                                 8    
source_degree                                                      
0              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                                                  \
target_degree                                                 9    
source_degree                                                      
0              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                     ...                          \
target_degree                        ...                           
source_degree                        ...                           
0                                    ...                           
1                                    ...                           

                                                                  \
target_degree                                                 14   
source_degree                                                      
0              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1              [0.0, 0.0, 0.0, 0.0, 0

In [9]:
mean_dgp = degree_grouped_permutations.applymap(np.mean)
std_dgp = degree_grouped_permutations.applymap(np.std)
num_dgp = degree_grouped_permutations.applymap(len)